In [127]:
"""
voc (int): xmin,ymin,xmax,ymax
yolo (float): cx,cy,w,h
coco (int): xmin,ymin,w,h
"""
import os, glob, re, json
import cv2 # yolo2others only

def voc2yolo(sourceFolder, destFolder, classList):
    with open(destFolder+"/classes.txt","w") as f:
        for c in classList:
            f.writelines(c+"\n")
    for xmlPath in glob.glob(sourceFolder+"/*.xml"):
        xml = open(xmlPath,"r").read()
        width  = int(re.findall("<width>([0-9]*)</width>",xml)[0])
        height = int(re.findall("<height>([0-9]*)</height>",xml)[0])
        nameL = re.findall("<name>(.*)</name>",xml)
        xminL = re.findall("<xmin>(.*)</xmin>",xml)
        yminL = re.findall("<ymin>(.*)</ymin>",xml)
        xmaxL = re.findall("<xmax>(.*)</xmax>",xml)
        ymaxL = re.findall("<ymax>(.*)</ymax>",xml)
        with open(destFolder+"/"+xmlPath.split('/')[-1].replace('.xml','.txt'), "w") as f:
            for name,xmin,ymin,xmax,ymax in zip(nameL,xminL,yminL,xmaxL,ymaxL):
                id = classList.index(name)
                cx = round((int(xmin)+int(xmax))/2/width,6)
                cy = round((int(ymin)+int(ymax))/2/height,6)
                w  = round((int(xmax)-int(xmin))/width,6)
                h  = round((int(ymax)-int(ymin))/height,6)
                pad = lambda s:str(s)+'0'*(8-len(str(s)))
                f.writelines(f"{id} {pad(cx)} {pad(cy)} {pad(w)} {pad(h)}\n")
#voc2yolo("./source", ".", ["dog","cat"])

def yolo2voc(sourceFolder, classesPath, destFolder):
    xml0="\
<annotation>\n\
	<folder>folderX</folder>\n\
	<filename>filenameX</filename>\n\
	<path>pathX</path>\n\
	<source>\n\
		<database>Unknown</database>\n\
	</source>\n\
	<size>\n\
		<width>widthX</width>\n\
		<height>heightX</height>\n\
		<depth>3</depth>\n\
	</size>\n\
	<segmented>0</segmented>\n\
" # folderX,filenameX,pathX,widthX,heightX need to be replaced
    obj0="\
	<object>\n\
		<name>nameX</name>\n\
		<pose>Unspecified</pose>\n\
		<truncated>0</truncated>\n\
		<difficult>0</difficult>\n\
		<bndbox>\n\
			<xmin>xminX</xmin>\n\
			<ymin>yminX</ymin>\n\
			<xmax>xmaxX</xmax>\n\
			<ymax>ymaxX</ymax>\n\
		</bndbox>\n\
	</object>\n\
" # nameX,xmin,ymin,xmax,ymax need to be replaced
    end0="</annotation>"
    with open(classesPath,"r") as f:
        D = {str(i):key[:-1] for i,key in enumerate(f.readlines())}
    txtPathL = glob.glob(sourceFolder+"/*.txt")   
    for txtPath in filter(lambda s:'classes.txt' not in s, txtPathL):
        folder   = destFolder.split('/')[-1] if "/" in destFolder else os.getcwd().split('/')[-1]
        filename = txtPath.split('/')[-1].replace('.txt','.jpg')
        path     = os.path.abspath(destFolder)+"/"+filename
        height, width, _ = cv2.imread(txtPath.replace('.txt','.jpg')).shape
        xml = xml0.replace('folderX',folder).replace('filenameX',filename).replace('pathX',path).\
            replace('widthX',str(width)).replace('heightX',str(height))
        for yoloLine in open(txtPath).readlines():
            id, cx, cy, w, h = yoloLine.split(" ")
            #print(id, cx, cy, w, h)
            xmin = round((float(cx)-float(w)/2)*width)
            ymin = round((float(cy)-float(h)/2)*height)
            xmax = round((float(cx)+float(w)/2)*width)
            ymax = round((float(cy)+float(h)/2)*height)
            #print(xmin, ymin, xmax, ymax)
            obj = obj0.replace('nameX',D[id]).replace('xminX',str(xmin)).replace('yminX',str(ymin)).\
                replace('xmaxX',str(xmax)).replace('ymaxX',str(ymax))
            xml+=obj
        xml+=end0
        with open(destFolder+"/"+filename.replace('.jpg','.xml'),'w') as f:
            f.write(xml)
#yolo2voc("source","./source/classes.txt",".")

def voc2coco(sourceFolder, destFolder, classList):
    """
    images0 = {"file_name":"X","height":-1,"width":-1,"id":"X"} # X->image name # -1->int
    annotations0 = {"area":0,"iscrowd":0,"bbox":[-1,-1,-1,-1],"category_id":-1,"ignore":0,"segmentation":[],\
        "image_id":"X","id":-1} # -1:int # X:image name # id:bbox-id per image # annot.image_id=images.id !!!
    categories0 = {"supercategory":"none", "id":-1, "name":"X0"} # id:categoryID # name:categoryName
    """
    D = {"images":[], "annotations":[], "categories": []}
    D["categories"] = [ {"supercategory":"none","id":i,"name":cat} for i,cat in enumerate(classList,1) ]
    for xmlPath in glob.glob(sourceFolder+"/*.xml"):
        xml = open(xmlPath,"r").read()
        filename = xmlPath.split('/')[-1].replace('.xml','.jpg')
        width  = int(re.findall("<width>([0-9]*)</width>",xml)[0])
        height = int(re.findall("<height>([0-9]*)</height>",xml)[0])
        nameL  = re.findall("<name>(.*)</name>",xml)
        xminL  = re.findall("<xmin>(.*)</xmin>",xml)
        yminL  = re.findall("<ymin>(.*)</ymin>",xml)
        xmaxL  = re.findall("<xmax>(.*)</xmax>",xml)
        ymaxL  = re.findall("<ymax>(.*)</ymax>",xml)
        D["images"].append( {"file_name":filename,"height":height,"width":width,"id":filename} )
        for i,(name,xmin,ymin,xmax,ymax) in enumerate(zip(nameL,xminL,yminL,xmaxL,ymaxL),1):
            xmin = int(xmin)
            ymin = int(ymin)
            w    = int(xmax)-int(xmin)
            h    = int(ymax)-int(ymin)
            catid= classList.index(name)+1
            D["annotations"].append( {"area":w*h,"iscrowd":0,"bbox":[xmin,ymin,w,h],"category_id":catid,\
                "ignore":0,"segmentation":[],"image_id":filename,"id":i} )
    with open(destFolder+"/label.json", "w") as f:
        json.dump(D,f)

voc2coco("./source", ".", ["dog","cat"])